# AI for Ethics
Final Project Summer 2020

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile

In [3]:
path_to_zip_file = './ny_hmda_2015.csv.zip'
directory_to_extract_to = './'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [4]:
df = pd.read_csv(filepath_or_buffer='./ny_hmda_2015.csv')

In [5]:
df.head()

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_abbr,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income
0,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,97.0,5,NaN,...,NY,New York,109000.0,187,363.0,1817.0,21.139999,5870.0,NaN,109.459999
1,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,200.0,5,NaN,...,NY,New York,71300.0,460,53.0,256.0,45.959999,3512.0,NaN,160.600006
2,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,NaN,3,NaN,...,NY,New York,71300.0,296,2745.0,2586.0,38.990002,8357.0,NaN,134.820007
3,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,202.0,5,NaN,...,NY,New York,109000.0,770,1879.0,2147.0,7.350000,6642.0,NaN,165.830002
4,1,Loan originated,9,CFPB,Consumer Financial Protection Bureau,2,Not Hispanic or Latino,255.0,5,NaN,...,NY,New York,109000.0,648,835.0,676.0,30.059999,2339.0,NaN,133.300003


In [6]:
df.shape

(439654, 78)

In [7]:
df.columns

Index(['action_taken', 'action_taken_name', 'agency_code', 'agency_abbr',
       'agency_name', 'applicant_ethnicity', 'applicant_ethnicity_name',
       'applicant_income_000s', 'applicant_race_1', 'applicant_race_2',
       'applicant_race_3', 'applicant_race_4', 'applicant_race_5',
       'applicant_race_name_1', 'applicant_race_name_2',
       'applicant_race_name_3', 'applicant_race_name_4',
       'applicant_race_name_5', 'applicant_sex', 'applicant_sex_name',
       'application_date_indicator', 'as_of_year', 'census_tract_number',
       'co_applicant_ethnicity', 'co_applicant_ethnicity_name',
       'co_applicant_race_1', 'co_applicant_race_2', 'co_applicant_race_3',
       'co_applicant_race_4', 'co_applicant_race_5',
       'co_applicant_race_name_1', 'co_applicant_race_name_2',
       'co_applicant_race_name_3', 'co_applicant_race_name_4',
       'co_applicant_race_name_5', 'co_applicant_sex', 'co_applicant_sex_name',
       'county_code', 'county_name', 'denial_reason_1', 